# **Principios SOLID en POO**

SOLID es un conjunto de cinco principios de diseño en **programación orientada a objetos (POO)** que buscan mejorar la calidad, mantenibilidad y escalabilidad del código. Cada letra representa un principio:

- **S - Single Responsibility Principle (SRP):** Cada clase debe tener una única responsabilidad o razón de cambio. Es decir, una clase debe encargarse de una sola cosa.

- **O - Open/Closed Principle (OCP):** El código debe estar abierto para la extensión, pero cerrado para la modificación. Esto significa que se deben poder agregar nuevas funcionalidades sin alterar el código existente, por ejemplo, usando **herencia** o **interfaces**.

- **L - Liskov Substitution Principle (LSP):** Los objetos de una subclase deben poder reemplazar a los objetos de su clase base sin alterar el funcionamiento del programa.

- **I - Interface Segregation Principle (ISP):** No se deben forzar a las clases a depender de interfaces que no usan. Es mejor tener varias **interfaces pequeñas y específicas** en lugar de una interfaz grande y genérica.

- **D - Dependency Inversion Principle (DIP):** El código debe depender de **abstracciones** y no de implementaciones concretas. Esto se logra mediante la **inyección de dependencias**, lo que facilita la reutilización y pruebas del código.


### 1. ***Single Responsibility Principle (SRP)***

✅ **Una clase debe tener una sola razón para cambiar.**

❌ **Violación del SRP**

Esta clase maneja tanto los datos del empleado como la impresión del reporte:


In [ ]:
import bcrypt  # Librería para cifrar contraseñas

class Ecommerce:
    """
    Clase que maneja tanto la lógica de negocio del ecommerce como la gestión de contraseñas.
    Esto viola el Principio de Responsabilidad Única (SRP).
    """
    
    def __init__(self):
        self.users = {}  # Diccionario para almacenar usuarios y contraseñas cifradas
        
    def register(self, username, password):
        """
        Registra un usuario y cifra la contraseña.
        Problema: La clase Ecommerce no debería encargarse del cifrado de contraseñas.
        """
        salt = bcrypt.gensalt()  
        hashed_password = bcrypt.hashpw(password.encode(), salt)  # 🚨 Rompe SRP
        self.users[username] = hashed_password  
        print(f"Usuario {username} registrado con éxito")  

# Crear instancia y registrar un usuario
ecommerce = Ecommerce()
ecommerce.register("Juan", "123")


Usuario Juan registrado con exito


#### ✅ ***Aplicando SRP***

Separamos la lógica de los datos y la impresión del reporte:

In [ ]:
import bcrypt  # Librería para cifrar contraseñas

class PasswordManager:
    """Maneja el cifrado y verificación de contraseñas, separando responsabilidades."""
    
    def encrypt_password(self, password: str) -> str:
        salt = bcrypt.gensalt()
        return bcrypt.hashpw(password.encode(), salt)
    
    def verify_password():
        pass  # Método pendiente de implementación

class Ecommerce:
    """Gestiona el registro de usuarios sin encargarse del cifrado, respetando SRP."""
    
    def __init__(self, password_manager: PasswordManager):
        self.users = {}  # Almacena usuarios con sus contraseñas cifradas
        self.password_manager = password_manager  # Delegación de la gestión de contraseñas
        
    def register(self, username, password):
        """Registra un usuario con su contraseña cifrada mediante PasswordManager."""
        hashed_password = self.password_manager.encrypt_password(password)
        self.users[username] = hashed_password
        print(f"Usuario {username} registrado con éxito")

# Crear instancia y registrar un usuario
password_manager = PasswordManager()  
ecommerce = Ecommerce(password_manager)
ecommerce.register("Juan", "123")


Usuario Juan registrado con exito


### ***2. Open/Closed Principle (OCP)***
✅ **El código debe ser abierto para extensión, pero cerrado para modificación.**

#### ❌ ***Violación del OCP***

Cada vez que agregamos un nuevo tipo de empleado, modificamos la clase:

In [ ]:
class CalculadoraDeAreas:
    """
    🚨 Esta clase viola el Principio de Abierto/Cerrado (OCP).
    Cada vez que queramos agregar una nueva figura, debemos modificar el código existente.
    """
    
    def calcular_area(self, figura: str, **kwargs):
        if figura == "circulo":
            return 3.14 * (kwargs["radio"] ** 2)
        elif figura == "rectangulo":
            return kwargs["ancho"] * kwargs["alto"]
        # 🚨 Si agregamos más figuras, tenemos que modificar esta función, lo cual rompe OCP

# Uso de la calculadora
calculadora = CalculadoraDeAreas()
print(calculadora.calcular_area("circulo", radio=5))  # 78.5
print(calculadora.calcular_area("rectangulo", ancho=4, alto=6))  # 24


78.5
24


#### ✅ ***Aplicando OCP***

Usamos herencia y polimorfismo en lugar de modificar la clase original.

In [ ]:
from abc import ABC, abstractmethod

# ✅ Aplicamos el Principio de Abierto/Cerrado (OCP) usando polimorfismo y abstracción
class FiguraGeometrica(ABC):
    """Clase abstracta que define una interfaz para calcular el área."""
    
    @abstractmethod
    def calcular_area(self) -> float:
        pass  # Cada figura implementará su propio cálculo de área

# ✅ Cada figura geométrica se define en su propia clase sin modificar el código existente
class Circulo(FiguraGeometrica):
    def __init__(self, radio: float):
        self.radio = radio
        
    def calcular_area(self) -> float:
        return 3.14 * (self.radio ** 2)

class Rectangulo(FiguraGeometrica):
    def __init__(self, ancho: float, alto: float):
        self.ancho = ancho
        self.alto = alto
    
    def calcular_area(self) -> float:
        return self.ancho * self.alto

class Triangulo(FiguraGeometrica):
    def __init__(self, base: float, altura: float):
        self.base = base
        self.altura = altura
    
    def calcular_area(self) -> float:
        return 0.5 * self.base * self.altura

# ✅ La CalculadoraDeAreas ahora puede aceptar cualquier nueva figura sin modificar su código
class CalculadoraDeAreas:
    def calcular(self, figura: FiguraGeometrica) -> float:
        return figura.calcular_area()

# Crear una instancia de la calculadora
calculadora = CalculadoraDeAreas()

# Crear figuras y calcular sus áreas
circulo = Circulo(5)
rectangulo = Rectangulo(4, 6)
triangulo = Triangulo(3, 5)

print(f"Área del círculo: {calculadora.calcular(circulo)}")     # 78.5
print(f"Área del rectángulo: {calculadora.calcular(rectangulo)}") # 24
print(f"Área del triángulo: {calculadora.calcular(triangulo)}")   # 7.5


Area de un circulo: 78.5


### ***3. Liskov Substitution Principle (LSP)***
✅ Las subclases deben poder reemplazar a sus clases base sin alterar el comportamiento.

#### ❌ ***Violación del LSP***
Supongamos que tenemos una clase Vehiculo con un método acelerar().
Luego, creamos una subclase Bicicleta, pero como las bicicletas no tienen motor, acelerar con una bicicleta no tiene sentido.

In [ ]:
class Vehiculo:
    def acelerar(self):
        print("Aumentando velocidad")

class Coche(Vehiculo):
    def acelerar(self):
        print("El coche acelera con el motor")

# 🔴 Bicicleta hereda de Vehiculo, pero no debería
class Bicicleta(Vehiculo):
    def acelerar(self):
        raise NotImplementedError("Las bicicletas no tienen acelerador")

# 🔹 Prueba del código
def probar_vehiculo(vehiculo: Vehiculo):
    vehiculo.acelerar()

coche = Coche()
bicicleta = Bicicleta()

probar_vehiculo(coche)       # ✅ Correcto: "El coche acelera con el motor"
probar_vehiculo(bicicleta)   # 🔴 Error: NotImplementedError


El coche acelera con el motor


NotImplementedError: Las bicilciteas no tiene acelerador

#### ✅ ***Ejemplo corregido que respeta LSP***
Para solucionar el problema, separamos VehiculoConMotor y VehiculoSinMotor para que cada uno tenga métodos apropiados.

In [2]:
from abc import ABC, abstractmethod

# ✅ Creamos una clase base abstracta para cualquier tipo de vehículo
class Vehiculo(ABC):
    @abstractmethod
    def mover(self):
        pass

# ✅ Coche hereda de Vehiculo y puede implementar el método correctamente
class Coche(Vehiculo):
    def mover(self):
        print("El coche acelera con el motor")

# ✅ Bicicleta ahora es un Vehiculo, pero implementa un método diferente (pedalear en lugar de acelerar)
class Bicicleta(Vehiculo):
    def mover(self):
        print("La bicicleta avanza al pedalear")

# ✅ La función ahora se llama 'probar_movimiento' en lugar de 'probar_vehiculo' para ser más general
def probar_movimiento(vehiculo: Vehiculo):
    vehiculo.mover()

# 🔹 Probamos con instancias de cada clase
coche = Coche()
bicicleta = Bicicleta()

probar_movimiento(coche)      # ✅ "El coche acelera con el motor"
probar_movimiento(bicicleta)  # ✅ "La bicicleta avanza al pedalear"


El coche acelera con el motor
La bicicleta avanza al pedalear


#### ***✅ ¿Por qué este código respeta LSP?***
✔️ Ahora Bicicleta y Coche pueden sustituir correctamente a Vehiculo, sin generar errores.

✔️ Cada clase tiene solo los métodos que realmente necesita, sin implementar métodos inválidos.

✔️ El programa sigue funcionando si cambiamos un Vehiculo por otro.

### ***5. Dependency Inversion Principle (DIP)***
✅ **Depende de abstracciones, no de implementaciones concretas.**

#### ❌ ***Ejemplo que viola DIP***
Aquí tenemos un sistema donde la clase Notificador depende directamente de la clase EmailService.

In [ ]:
class EmailService:
    def enviar_email(self, mensaje: str):
        print(f"Enviando email: {mensaje}")

class Notificador:
    def __init__(self):
        self.email_service = EmailService()  # 🔴 DEPENDENCIA DIRECTA 

    def notificar(self, mensaje: str):
        self.email_service.enviar_email(mensaje)  # 🔴 Acoplado a EmailService

# 🔹 Prueba del código
notificador = Notificador()
notificador.notificar("Hola, este es un mensaje importante")


Enviar email: Hola, Somos Dev Senior


#### ✅ ***Ejemplo corregido aplicando DIP***
Para solucionarlo, usamos una abstracción (INotificador) que define el comportamiento general y luego creamos implementaciones concretas (EmailService, SMSService, etc.).

In [ ]:
from abc import ABC, abstractmethod

# ✅ Interfaz para cumplir con DIP (Principio de Inversión de Dependencias)
class INotificacion(ABC):
    @abstractmethod
    def enviar(self, mensaje: str):
        pass
    
# ✅ Clases de servicios de notificación que implementan la interfaz (cumple OCP)
class EmailServicie(INotificacion):
    def enviar(self, mensaje: str):
        print(f"Enviar email: {mensaje}")
        
class SMSServicie(INotificacion):
    def enviar(self, mensaje):
        print(f"Enviar SMS: {mensaje}")
        
class WhatAppServicie(INotificacion):
    def enviar(self, mensaje):
        print(f"Enviar WhatsApp: {mensaje}")

# ✅ Notificador desacoplado de servicios específicos (cumple DIP)
class Notificardor:
    def __init__(self, servicio: INotificacion):
        self.servicio = servicio
        
    def notificar(self, mensaje: str):
        self.servicio.enviar(mensaje)

# ✅ Clase adicional que permite modificar mensajes sin alterar Notificador (cumple SRP)
class NotificadorModificable:
    def __init__(self, notificador: Notificardor):
        self.notificacion = notificador
        self.mensaje = None  # Variable para almacenar el mensaje actual
        
    def notificar(self, mensaje: str):
        """Envía una notificación y almacena el mensaje."""
        self.mensaje = mensaje
        self.notificacion.notificar(mensaje)
        
    def modificar_mensaje(self, nuevo_mensaje: str):
        """Modifica el mensaje previo y lo reenvía."""
        if self.mensaje is None:
            print("No hay mensaje previo para modificar")
            return
        self.mensaje = nuevo_mensaje
        print(f"Mensaje modificado a: {self.mensaje}")
        self.notificacion.notificar(self.mensaje)

# ✅ Crear instancias con inyección de dependencias (cumple DIP)
email_notificador = NotificadorModificable(Notificardor(EmailServicie()))  
sms_notificador = Notificardor(SMSServicie())
whatApp_notificador = Notificardor(WhatAppServicie())

# 📌 Mensaje Inicial
email_notificador.notificar("Hola, somos Dev Senior desde Email")
sms_notificador.notificar("Hola, somos Dev Senior desde SMS")
whatApp_notificador.notificar("Hola, somos Dev Senior desde WhatsApp")

# 📌 Modificar y reenviar mensaje (solo en Email)
email_notificador.modificar_mensaje("Hola, somos Dev Senior con mensaje modificado")


Enviando email: Hola, este es un mensaje por Email
Enviando SMS: Hola, este es un mensaje por SMS


#### ✅ ***¿Por qué este código respeta DIP?***
✔️ Notificador ya no depende de una clase concreta (EmailService o SMSService), sino de la interfaz INotificador.

✔️ Podemos agregar nuevas implementaciones (ej. WhatsApp, Telegram, etc.) sin modificar Notificador.

✔️ El código es más flexible y abierto a la extensión sin afectar las clases existentes.

### ***Conclusión sobre SOLID y su importancia para un desarrollador***

Los principios **SOLID** son fundamentales para escribir código **limpio, mantenible y escalable**. Cada uno de estos principios aborda problemas comunes en el desarrollo de software, permitiendo que las aplicaciones sean más **modulares, reutilizables** y fáciles de extender sin necesidad de modificar el código existente.

#### ***Importancia de SOLID para un desarrollador:***

- **Facilita el mantenimiento del código:**
  Aplicar SOLID reduce la complejidad y facilita la localización y corrección de errores sin afectar otras partes del sistema.

- **Promueve la reutilización de código:**
  Diseñar clases y módulos con responsabilidades bien definidas permite reutilizar componentes en diferentes partes de un proyecto o en futuros desarrollos.

- **Mejora la escalabilidad del software:**
  Un código bien estructurado según SOLID es más fácil de extender con nuevas funcionalidades sin necesidad de modificar el código existente, evitando la introducción de nuevos errores.

- **Reduce el acoplamiento y mejora la modularidad:**
  Siguiendo SOLID, cada clase y módulo depende solo de lo estrictamente necesario, evitando dependencias innecesarias y facilitando la prueba y depuración del código.

- **Fomenta las buenas prácticas de diseño:**
  Aplicar SOLID ayuda a adoptar un enfoque de desarrollo más profesional, siguiendo estándares que facilitan el trabajo en equipo y la colaboración en proyectos grandes.

**SOLID no es solo un conjunto de reglas, sino una guía para escribir mejor código.**
Todo desarrollador que quiera crear software de calidad debería aplicar estos principios para lograr sistemas más **robustos, flexibles y fáciles de mantener**. 🚀
